# Import des module

In [1]:
import json
import psycopg2 as pg 
import datetime as dt 

# config de base et lecture fichier

## ouverture connexion

In [2]:
conn = pg.connect(database="vol",
                        host="127.0.0.1",
                        user="postgres",
                        password="postgres",
                        port="5432")

In [3]:
def execute_query(query, conn = conn):
    cursor = conn.cursor()
    cursor.execute(query)
    cursor.close()
    conn.commit()

## lecture et exution file create_work_db

In [4]:
with open("create_work_db.sql", "r")  as work_db:
    lines = work_db.readlines()

query = ""
for i in lines:
    i = i.strip()
    query += i

cursor = conn.cursor()
cursor.execute(query)
cursor.close()
conn.commit()

## lecture file Json et recup de results

In [5]:
with open("200_751adea7_edfc_4626_acdb_adbe32336a33.log", "r") as file : 
    data = json.load(file)

data = data["content"]["results"]

#  traitement table 

## aliance

In [6]:
alliance = data['alliances']

query = "INSERT INTO WORK_ALLIANCE (id_alliance, code_alliance, name) VALUES "
i = 1
for key, item in alliance.items():
    code_alliance = int(key)
    name = item['name']
    query += f"({i}, {code_alliance}, '{name}'),"
    i += 1

query = query[:-1] + ";"
execute_query(query)

query = "INSERT INTO ALLIANCE (code_alliance, name) SELECT code_alliance, name FROM WORK_ALLIANCE WHERE NOT EXISTS ( SELECT 1 FROM ALLIANCE WHERE ALLIANCE.code_alliance = WORK_ALLIANCE.code_alliance );"

execute_query(query)

## carriere

### remplisage sans id alliance (avec code_alliance)

In [7]:
carriere = data['carriers']

query = "INSERT INTO WORK_CARRIERE (id_carriere, code_carriere, code_alliance,img_url, iata, icao, displaycode, name) VALUES "

i = 1
for key, item in carriere.items():
    code_carriere = int(key)
    name = item['name']
    try:
        alliance_code = int(item['allianceId'])
    except:
        alliance_code = "NULL"
    img_url = item['imageUrl']
    iata = item['iata']
    icao = item['icao']
    displaycode = item['displayCode']
    
    query += f"({i}, {code_carriere}, {alliance_code}, '{img_url}', '{iata}', '{icao}', '{displaycode}', '{name}'),"
    i += 1

query = query[:-1] + ";"
execute_query(query)

### jointure pour recuperer id

In [8]:
query = "INSERT INTO WORK_CARRIERE2 (id_carriere, code_carriere, id_alliance,img_url, iata, icao, displaycode, name) SELECT id_carriere, code_carriere, id_alliance, img_url, iata, icao, displaycode, wc.name FROM ALLIANCE wa RIGHT JOIN WORK_CARRIERE wc on wa.code_alliance = wc.code_alliance;"

execute_query(query)

query = "INSERT INTO CARRIERE (code_carriere, id_alliance,img_url, iata, icao, displaycode, name) SELECT code_carriere, id_alliance,img_url, iata, icao, displaycode, name FROM WORK_CARRIERE2 WHERE NOT EXISTS (SELECT 1 FROM CARRIERE WHERE CARRIERE.code_carriere = WORK_CARRIERE2.code_carriere);"
execute_query(query)

## country

In [9]:
places = data['places']

query = "INSERT INTO WORK_COUNTRY (id_country, code_country, name, coordinate) VALUES "

i = 1
for key, item in places.items():
    if item['type'] == "PLACE_TYPE_COUNTRY":
        code_country = int(key)
        name = item['name']
        coordinate =  item["coordinates"] if item["coordinates"] != None else "NULL"
        query += f"({i}, {code_country}, '{name}', '{coordinate}'),"
        i += 1


query = query[:-1] + ";"
execute_query(query)

query = "INSERT INTO COUNTRY (code_country, name, coordinate) SELECT code_country, name, coordinate FROM WORK_COUNTRY WHERE NOT EXISTS (SELECT 1 FROM COUNTRY WHERE COUNTRY.code_country = WORK_COUNTRY.code_country);"
execute_query(query)

## city

### recup val

In [10]:
places = data['places']

query = "INSERT INTO WORK_CITY (id_city, code_city, name, coordinate, iata, code_country) VALUES "

i = 1
for key, item in places.items():
    if item['type'] == "PLACE_TYPE_CITY":
        code_city = int(key)
        name = item['name']
        coordinate =  item["coordinates"] if item["coordinates"] != None else "NULL"
        iata = item['iata']
        code_country = int(item['parentId'])
        query += f"({i}, {code_city}, '{name}', '{coordinate}', '{iata}', {code_country}),"
        i += 1


query = query[:-1] + ";"
execute_query(query)

### join for get id_country

In [11]:
query = "INSERT INTO WORK_CITY2 (id_city, code_city, name, coordinate, iata, id_country) SELECT id_city, code_city, wc.name, wc.coordinate, iata, id_country FROM WORK_CITY wc LEFT JOIN COUNTRY w on wc.code_country = w.code_country;"
execute_query(query)

query = "INSERT INTO CITY (code_city, name, coordinate, iata, id_country) SELECT code_city, name, coordinate, iata, id_country FROM WORK_CITY2 WHERE NOT EXISTS (SELECT 1 FROM CITY WHERE CITY.code_city = WORK_CITY2.code_city);"
execute_query(query)

## airport

### get val without id_city

In [12]:
places = data['places']

query = "INSERT INTO WORK_AIRPORT (id_airport, code_airport, name, coordinate, iata, code_city) VALUES "

i = 1
for key, item in places.items():
    if item['type'] == "PLACE_TYPE_AIRPORT":
        code_airport = int(key)
        name = item['name']
        coordinate = item["coordinates"] if item["coordinates"] != None else "NULL"
        iata = item['iata']
        code_city = int(item['parentId'])
        query += f"({i}, {code_airport}, '{name}', '{coordinate}', '{iata}', {code_city}),"
        i += 1


print(query)
query = query[:-1] + ";"
execute_query(query)

INSERT INTO WORK_AIRPORT (id_airport, code_airport, name, coordinate, iata, code_city) VALUES (1, 95565051, 'Londres Gatwick', 'NULL', 'LGW', 27544008),(2, 95565070, 'Milan Malpensa', 'NULL', 'MXP', 27544068),(3, 95565077, 'Madrid-Barajas', 'NULL', 'MAD', 27544850),(4, 95673529, 'Dublin', 'NULL', 'DUB', 27540823),(5, 95673588, 'Marrakech Menara', 'NULL', 'RAK', 27546125),(6, 95673860, 'Nice', 'NULL', 'NCE', 27545091),(7, 95673976, 'Bordeaux', 'NULL', 'BOD', 27539513),


### get id city

In [13]:
query = "INSERT INTO WORK_AIRPORT2 (id_airport, code_airport, name, coordinate, iata, id_city) SELECT id_airport, code_airport, wc.name, wc.coordinate, wc.iata, id_city FROM WORK_AIRPORT wc LEFT JOIN CITY w on wc.code_city = w.code_city;"
execute_query(query)

query = "INSERT INTO AIRPORT (code_airport, name, coordinate, iata, id_city) SELECT code_airport, name, coordinate, iata, id_city FROM WORK_AIRPORT2 WHERE NOT EXISTS (SELECT 1 FROM AIRPORT WHERE AIRPORT.code_airport = WORK_AIRPORT2.code_airport);"
execute_query(query)

## segment et date

### without id

In [14]:
segment = data['segments']

query = "INSERT INTO WORK_SEGMENT (id_segment, code_segment, code_airport_depart,  code_airport_arrive, depart, arrive, marketing_carriere_id, operational_carriere_id, marketing_flight_number, duration_minute) VALUES "
t_arr = []
t_dep = []
i = 1
for key, item in segment.items():
    code_segment = key
    code_airport_depart = item['originPlaceId']
    code_airport_arrive = item['destinationPlaceId']
    dep = item['departureDateTime']
    arr = item['arrivalDateTime']
    depart = dt.datetime(dep['year'], dep['month'], dep['day'], dep['hour'], dep['minute'], dep['second']).timestamp()
    arrive = dt.datetime(arr['year'], arr['month'], arr['day'], arr['hour'], arr['minute'], arr['second']).timestamp()
    duration = item['durationInMinutes']
    marketing_flight_number = item['marketingFlightNumber']
    marketing_carrer_id = item['marketingCarrierId']
    operatin_carrier_id = item['operatingCarrierId']
    t_dep.append(depart)
    t_arr.append(arrive)
    query += f"({i}, '{code_segment}', {code_airport_depart}, {code_airport_arrive}, TO_TIMESTAMP({depart}), TO_TIMESTAMP({arrive}), {marketing_carrer_id}, {operatin_carrier_id}, {marketing_flight_number}, {duration}),"
    i += 1

t_arr = list(set(t_arr))
t_dep = list(set(t_dep))

query_dep = "INSERT INTO WORK_DEPART (id_depart, date) VALUES "
i = 1
for date in t_dep:
    query_dep += f"({i}, TO_TIMESTAMP({date})),"
    i += 1


query_arr = "INSERT INTO WORK_ARRIVER (id_arriver, date) VALUES "
i = 1
for date in t_arr:
    query_arr += f"({i}, TO_TIMESTAMP({date})),"
    i += 1

query = query[:-1] + ";"
query_arr = query_arr[:-1] + ";"
query_dep = query_dep[:-1] + ";"

execute_query(query)
execute_query(query_arr)
execute_query(query_dep)

### get id

In [15]:
query = """
INSERT INTO DEPART (date)
SELECT date
FROM WORK_DEPART
WHERE NOT EXISTS (
    SELECT 1 
    FROM DEPART 
    WHERE DEPART.date = WORK_DEPART.date
);

INSERT INTO ARRIVER (date)
SELECT date
FROM WORK_ARRIVER
WHERE NOT EXISTS (
    SELECT 1 
    FROM ARRIVER 
    WHERE ARRIVER.date = WORK_ARRIVER.date
);
"""

execute_query(query)

query = """
INSERT INTO SEGMENT (code_segment, id_airport_depart, id_airport_arrive, id_depart, id_arrive, marketing_carriere_id, operational_carriere_id, marketing_flight_number, duration_minute)
SELECT ws.code_segment, wa2.id_airport, wa22.id_airport, wd.id_depart, wa.id_arriver, wc2.id_carriere, wc.id_carriere, ws.marketing_flight_number, ws.duration_minute
FROM WORK_SEGMENT ws 
LEFT JOIN DEPART wd on ws.depart = wd.date
LEFT JOIN ARRIVER wa on ws.arrive = wa.date
LEFT JOIN AIRPORT wa2 on ws.code_airport_depart = wa2.code_airport
LEFT JOIN AIRPORT wa22 on ws.code_airport_arrive = wa22.code_airport
LEFT JOIN CARRIERE wc on ws.operational_carriere_id = wc.code_carriere
LEFT JOIN CARRIERE wc2 on ws.marketing_carriere_id = wc2.code_carriere
WHERE NOT EXISTS (
    SELECT 1
    FROM SEGMENT
    WHERE SEGMENT.code_segment = ws.code_segment
);
"""

execute_query(query)

## agent

In [16]:
agent = data['agents']

query = "INSERT INTO WORK_AGENT (id_agent, code_agent, name,type, img_url, feedback_count, rating, is_optimise_formability, custom_service, reliable_price, clear_extract_fees, easy_of_booking, other) VALUES "

i = 1
for key, item in agent.items():
    name = item['name']
    type_a = item['type']
    img = item['imageUrl']
    feedback = item['feedbackCount']
    rating = item['rating']
    is_optime = item['isOptimisedForMobile']
    rating_b = item['ratingBreakdown']
    custom_service = rating_b['customerService'] if rating_b != None else 'NULL'
    reliable_price = rating_b['reliablePrices'] if rating_b != None else 'NULL'
    clear_extract = rating_b['clearExtraFees'] if rating_b != None else 'NULL'
    easy_of = rating_b['easeOfBooking'] if rating_b != None else 'NULL'
    other = rating_b['other'] if rating_b != None else 'NULL'
    query += f"({i}, '{key}', '{name}', '{type_a}', '{img}', {feedback}, {rating}, {is_optime}, {custom_service}, {reliable_price}, {clear_extract}, {easy_of}, {other}),"
    i += 1


query = query[:-1] + ";"
execute_query(query)

query = """
BEGIN TRANSACTION;
MERGE INTO AGENT AS cible
USING  WORK_AGENT AS source
ON cible.code_agent = source.code_agent AND end_at IS NULL 
AND (cible.name != source.name OR 
    cible.type = source.type OR
    cible.img_url = source.img_url OR
    cible.feedback_count != source.feedback_count OR 
    cible.rating != source.rating OR
    cible.is_optimise_formability != source.is_optimise_formability OR
    cible.custom_service != source.custom_service OR 
    cible.reliable_price != source.reliable_price OR 
    cible.clear_extract_fees != source.clear_extract_fees OR 
    cible.easy_of_booking != source.easy_of_booking OR 
    cible.other != source.other )
WHEN MATCHED THEN
    UPDATE SET end_at = CURRENT_DATE;
    

COMMIT;

INSERT INTO AGENT (code_agent, name,type, img_url, feedback_count, rating, is_optimise_formability, custom_service, reliable_price, clear_extract_fees, easy_of_booking, other, start_at)
SELECT source.code_agent, source.name,source.type, source.img_url, source.feedback_count, source.rating, source.is_optimise_formability, source.custom_service, source.reliable_price, source.clear_extract_fees, source.easy_of_booking, source.other, CURRENT_DATE
FROM WORK_AGENT AS source
WHERE NOT EXISTS (
    SELECT 1 
    FROM AGENT AS cible
    WHERE cible.code_agent = source.code_agent 
	AND cible.end_at IS NULL 
);
"""

execute_query(query)

## leg

In [17]:
query_leg = "INSERT INTO WORK_LEG (code_leg, airport_depart, airport_arrive, depart, arrive, stop_count, duration_minute) VALUES "
query_seg_leg = "INSERT INTO WORK_SEGMENT_LEG (code_leg, code_segment) VALUES "

leg = data['legs']
for key, item in leg.items(): 
    arrive = dt.datetime(item['arrivalDateTime']['year'], item['arrivalDateTime']['month'], item['arrivalDateTime']['day'], item['arrivalDateTime']['hour'], item['arrivalDateTime']['minute'], item['arrivalDateTime']['second']).timestamp()
    depart = dt.datetime(item['departureDateTime']['year'], item['departureDateTime']['month'], item['departureDateTime']['day'], item['departureDateTime']['hour'], item['departureDateTime']['minute'], item['departureDateTime']['second']).timestamp()
    query_leg += f"('{key}', {item['originPlaceId']}, {item['destinationPlaceId']}, TO_TIMESTAMP({depart}), TO_TIMESTAMP({arrive}), {item['stopCount']}, {item['durationInMinutes']}),"

    for i in item['segmentIds']: 
        query_seg_leg += f"('{key}', '{i}'),"

query_leg = query_leg[:-1] + ";"
query_seg_leg = query_seg_leg[:-1] + ";"

execute_query(query_leg)
execute_query(query_seg_leg)

### final request for leg en seg_leg

In [18]:
query = """
INSERT INTO LEG (code_leg, airport_depart, airport_arrive, depart, arrive, stop_count, duration_minute)
SELECT code_leg, airport_depart, airport_arrive, depart, arrive, stop_count, duration_minute
FROM WORK_LEG
WHERE NOT EXISTS (
    SELECT 1 
    FROM LEG 
    WHERE LEG.code_leg = WORK_LEG.code_leg
);
"""
execute_query(query)

query = """
INSERT INTO SEGMENT_LEG (id_leg, id_segment)
SELECT l.id_leg, s.id_segment FROM WORK_SEGMENT_LEG wsl JOIN LEG l ON wsl.code_leg = l.code_leg JOIN SEGMENT s ON wsl.code_segment = s.code_segment WHERE NOT EXISTS (
    SELECT 1 FROM SEGMENT_LEG sl WHERE sl.id_leg = l.id_leg AND sl.id_segment = s.id_segment
);
"""

execute_query(query)

## itineraire, pricing option, pricing fare, seg leg

In [19]:
itineraires = data['itineraries']
query_itineraire = "INSERT INTO WORK_ITINERAIRE (code_itineraire, sustainable_data) VALUES "
query_pricing_option = "INSERT INTO WORK_PRICINGOPTION (code_itineraire, amount, unit, status, tranfert_type, id, pricing_option_fare) VALUES "
query_pricing_fare = "INSERT INTO WORK_PRICINGFARE (code_agent, amount, unit, update_status, deeplink, booking_code, fare_basis_code, code_segment) VALUES "
query_seg_leg = "INSERT INTO WORK_ITINERAIRE_LEG (code_itineraire, code_leg) VALUES "

for key, itineraire in itineraires.items():
    query_itineraire += f"('{key}', '{itineraire['sustainabilityData']}'),"

    for item in itineraire['legIds']: 
        query_seg_leg += f"('{key}', '{item}'),"

    for j, po in enumerate(itineraire['pricingOptions'], start=1):
        query_pricing_option += f"""('{key}', {po['price']['amount']}, '{po['price']['unit']}', '{po['price']['updateStatus']}', '{po['transferType']}', '{po['id']}', '{po['pricingOptionFare']}'),"""

        for k, pf in enumerate(po['items'], start=1):
            template = f"""('{pf['agentId']}', {pf['price']['amount']}, '{pf['price']['unit']}', '{pf['price']['updateStatus']}', '{pf['deepLink']}',"""
            for item in pf['fares']:
                query_pricing_fare = query_pricing_fare + template + f"""'{item['bookingCode']}', '{item['fareBasisCode']}', '{item['segmentId']}'),"""

    

query_itineraire = query_itineraire[:-1] + ";"
query_pricing_option = query_pricing_option[:-1] + ";"
query_pricing_fare = query_pricing_fare[:-1] + ";"
query_seg_leg = query_seg_leg[:-1] + ";"

execute_query(query_itineraire)
execute_query(query_pricing_option)
execute_query(query_pricing_fare)
execute_query(query_seg_leg)

### final request for itinerary, prising option, prising fare,leg_itinerary

In [20]:
query = """
INSERT INTO ITINERAIRE (code_itineraire, sustainable_data)
SELECT code_itineraire, sustainable_data
FROM WORK_ITINERAIRE
WHERE NOT EXISTS (
    SELECT 1 
    FROM ITINERAIRE 
    WHERE ITINERAIRE.code_itineraire = WORK_ITINERAIRE.code_itineraire
);
"""
execute_query(query)

query = """
INSERT INTO ITINERAIRE_LEG (id_leg, id_itineraire)
SELECT l.id_leg, i.id_itineraire FROM WORK_ITINERAIRE_LEG sil JOIN LEG l ON sil.code_leg = l.code_leg JOIN ITINERAIRE i ON sil.code_itineraire = i.code_itineraire WHERE NOT EXISTS (
    SELECT 1 FROM ITINERAIRE_LEG sl WHERE sl.id_leg = l.id_leg AND sl.id_itineraire = i.id_itineraire
);
"""
execute_query(query)

query = """
INSERT INTO PRICINGOPTION (id_itineraire, amount, unit, status, tranfert_type, id, pricing_option_fare)
SELECT id_itineraire, amount, unit, status, tranfert_type, id, pricing_option_fare
FROM WORK_PRICINGOPTION wpo JOIN ITINERAIRE i ON wpo.code_itineraire = i.code_itineraire;
"""
execute_query(query)

query = """
INSERT INTO PRICINGFARE (id_agent, amount, unit, update_status, deeplink, booking_code, fare_basis_code, id_segment)
SELECT id_agent, amount, unit, update_status, deeplink, booking_code, fare_basis_code, id_segment
FROM WORK_PRICINGFARE wpf JOIN SEGMENT s ON wpf.code_segment = s.code_segment JOIN AGENT a ON wpf.code_agent = a.code_agent;
"""
execute_query(query)

# close connexion

In [21]:
conn.close()